In [1]:
import os
PATHs = ['/p/lustre3/shan4/database/corpus', '/p/vast1/shan4/tmp-corpus', '/p/lustre1/shan4/tmp-corpus', '/usr/workspace/shan4/tar-gz']
both = []
source_nobc = []
bc_nosource = []
neither = []
directories_all = []
def find_directories_with_bc():
    directories_with_bc = []
    for start_path in PATHs:
        for subdir in next(os.walk(start_path))[1]:
            directories_all.append(subdir)
            full_subdir_path = os.path.join(start_path, subdir)
            for dirpath, dirnames, filenames in os.walk(full_subdir_path):
                for filename in filenames:
                    if filename.endswith('.bc'):
                        if subdir not in directories_with_bc:
                            directories_with_bc.append(subdir)
                        break
                else:
                    continue
                break
        

    return directories_with_bc

def find_directories_with_source():
    directories_with_bc = []
    for start_path in PATHs:
        for subdir in next(os.walk(start_path))[1]:
            directories_all.append(subdir)
            full_subdir_path = os.path.join(start_path, subdir)
            for dirpath, dirnames, filenames in os.walk(full_subdir_path):
                for filename in filenames:
                    if filename.endswith('.source'):
                        if subdir not in directories_with_bc:
                            directories_with_bc.append(subdir)
                        break
                else:
                    continue
                break
        

    return directories_with_bc

In [2]:
start_path = '/p/lustre3/shan4/database/corpus'
directories = find_directories_with_bc()
directories1 = find_directories_with_source() 

In [4]:
for i in directories_all:
    if (i not in directories1) and (i not in directories):
        neither.append(i)
    elif (i in directories1) and (i not in directories):
        source_nobc.append(i)
    elif (i not in directories1) and (i in directories):
        bc_nosource.append(i)
    else:
        both.append(i)

both = list(set(both))
len(both)

3236

In [65]:
import os



# Define list of subfolder names
subfolder_names = neither

# List to store contents
content_list = []
feedbacks = {}
for subfolder in subfolder_names:
    subfolder = subfolder.replace('/','_')
    found = False
    for mother_folder in PATHs:
        subfolder_path = os.path.join(mother_folder, subfolder)
        a_txt_path = os.path.join(subfolder_path, 'portage_build.log')
        
        if os.path.exists(a_txt_path):
            with open(a_txt_path, 'r') as file:
                content = file.read()
                content_list.append(content)
                feedbacks[subfolder] = content
            found = True
            break  # Stop searching once the subfolder is found and read
    
    if not found:
        print(f"Subfolder {subfolder} not found in any parent folder.")

Subfolder hello-2.11 not found in any parent folder.
Subfolder hello not found in any parent folder.
Subfolder x264 not found in any parent folder.


In [67]:
len(feedbacks)

5237

In [66]:
import json
with open('../ir-llvm/neither-data.json', 'w') as json_file:
    json.dump(feedbacks, json_file, indent=4)

In [21]:
installed = []
notinstalled = []
todo = []
with open('/p/lustre3/shan4/ir-llvm/corpus_descriptions_test/pkg.list','r') as pkglist:
    for pkg in pkglist:
        if pkg.replace('/','_')[:-1] in both:
            installed.append(pkg.replace('/','_')[:-1])
        elif pkg.replace('/','_')[:-1] in bc_nosource or pkg.replace('/','_')[:-1] in source_nobc:
            installed.append(pkg.replace('/','_')[:-1])
        elif pkg.replace('/','_')[:-1] in neither:
            # continue
            todo.append(pkg[:-1].replace('/','_')[:-1])
        else:
            #continue
            notinstalled.append(pkg[:-1].replace('/','_')[:-1])

In [22]:
import json
def analyse_neither():
    failed = []
    successed = []
    with open('../ir-llvm/portage-lists/neither.json') as neither_pkgs_files:
        neither_pkgs = json.load(neither_pkgs_files) 
    for pkg in neither_pkgs:
        if neither_pkgs[pkg] == 'N':
            failed.append(pkg)
        if neither_pkgs[pkg] == 'Y':
            successed.append(pkg)
    
    return failed, successed
failed, successed = analyse_neither()

In [39]:
for pkg in installed:
    if pkg not in bc_nosource and pkg not in source_nobc and pkg not in both:
        print(pkg)

In [41]:
finished = []
finished.extend(both)
finished.extend(bc_nosource)
finished.extend(source_nobc)
finished.extend(successed)
finished = list(set(finished))

In [48]:
with open('./portage-lists/finished.list', 'w') as todolist:
    for i in finished:
        todolist.write(i+'\n')